In [6]:
import pandas as pd
import numpy as np

### ----------------------------------------------------------
### 1. LOAD DATA
### ----------------------------------------------------------

MAIN_DIR = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite"
sing = pd.read_csv(MAIN_DIR + r"\singles_matches_clean.csv")

### ----------------------------------------------------------
### 2. CLEAN winner1 / loser1
### ----------------------------------------------------------

# Convert to numeric (fixes blank strings, NaN, or bad values)
sing["winner1"] = pd.to_numeric(sing["winner1"], errors="coerce")
sing["loser1"]  = pd.to_numeric(sing["loser1"], errors="coerce")

# Drop matches with missing or invalid IDs
sing = sing.dropna(subset=["winner1", "loser1"])
sing = sing[(sing["winner1"] > 0) & (sing["loser1"] > 0)]

# Convert to int now that they're clean
sing["winner1"] = sing["winner1"].astype(int)
sing["loser1"]  = sing["loser1"].astype(int)


### ----------------------------------------------------------
### 3. SCORE MARGIN (use set1, set2, set3)
### ----------------------------------------------------------

def score_to_margin(row):
    total = 0
    for col in ["set1", "set2", "set3"]:
        val = row[col]
        if isinstance(val, str) and "*" in val:
            a, b = val.split("*")
            total += (int(a) - int(b))
    return total

sing["margin"] = sing.apply(score_to_margin, axis=1)


### ----------------------------------------------------------
### 4. FLIGHT WEIGHTS
###    S1 > S2 > S3 > S4
### ----------------------------------------------------------

flight_weight = {"S1": 1.30, "S2": 1.15, "S3": 1.05, "S4": 1.00}
sing["flight_weight"] = sing["flight"].map(flight_weight).fillna(1.00)


### ----------------------------------------------------------
### 5. INITIALIZE RATINGS
### ----------------------------------------------------------

all_players = pd.unique(sing[["winner1", "loser1"]].values.ravel("K"))
ratings = {pid: 20.0 for pid in all_players}   # neutral starting point


### ----------------------------------------------------------
### 6. ELO FUNCTION
### ----------------------------------------------------------

def expected_score(rA, rB):
    # Smaller divisor → larger rating spread sensitivity (tennis appropriate)
    return 1 / (1 + 10**((rB - rA) / 12))


K = 6.0   # base update size


### ----------------------------------------------------------
### 7. ITERATIVE TRAINING
### ----------------------------------------------------------

for epoch in range(12):
    for _, row in sing.iterrows():

        w = row["winner1"]
        l = row["loser1"]
        margin = row["margin"]
        f = row["flight_weight"]

        rw = ratings[w]
        rl = ratings[l]

        Ew = expected_score(rw, rl)
        El = 1 - Ew

        margin_factor = 1 + (abs(margin) * 0.10)

        ratings[w] += K * f * margin_factor * (1 - Ew)
        ratings[l] += K * f * margin_factor * (0 - El)


### ----------------------------------------------------------
### 8. NORMALIZE TO RANGE 5–39
### ----------------------------------------------------------

rating_series = pd.Series(ratings)
min_r = rating_series.min()
max_r = rating_series.max()

def scale_5_39(x):
    return 5 + (x - min_r) * (34 / (max_r - min_r))

scaled = rating_series.apply(scale_5_39)


### ----------------------------------------------------------
### 9. BUILD FINAL OUTPUT TABLE
### ----------------------------------------------------------

player_ratings = pd.DataFrame({
    "playerID": scaled.index,
    "rating": scaled.values
}).sort_values("rating", ascending=False)

player_ratings.to_csv(MAIN_DIR + r"\player_ratings.csv", index=False)

player_ratings.head(20)

,playerID,rating
75,163631,39.000000
4,123420,37.108171
249,145172,37.057582
70,164183,36.752026
80,141982,36.271394
125,104618,35.204742
84,143078,34.785011
9,141640,34.604943
82,123263,34.273460
320,121899,34.163175


In [4]:
player_ratings.head(200)

,playerID,rating,fullname
662,121528,39.000000,NaN
647,142719,38.728271,NaN
661,141812,37.676313,NaN
660,162888,35.991784,NaN
664,162883,35.607230,NaN
...,...,...,...
111,121407,NaN,Lance Palm
112,122807,NaN,Reece Gregor
113,147511,NaN,Anna Poe
114,123748,NaN,WahDoh Thoo


In [5]:
print("Rows loaded:", len(sing))
print("Rows with missing winner1:", sing["winner1"].isna().sum())
print("Rows with missing loser1:", sing["loser1"].isna().sum())

print("Sample winner1 values:", sing["winner1"].head())
print("Sample loser1 values:", sing["loser1"].head())

print("Unique flights:", sing["flight"].unique())


Rows loaded: 4081
Rows with missing winner1: 0
Rows with missing loser1: 0
Sample winner1 values: 0    143027
1    171234
2    141639
3     99031
4    123420
Name: winner1, dtype: int32
Sample loser1 values: 0    165951
1    143025
2     99025
3    141570
4     99367
Name: loser1, dtype: int32
Unique flights: ['S1' 'S2' 'S3' 'S4' 'S5' 'S7' 'S10' 'S6']


In [7]:
### ----------------------------------------------------------
### 1. LOAD DATA
### ----------------------------------------------------------

MAIN_DIR = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite"
doub = pd.read_csv(MAIN_DIR + r"\doubles_matches_clean.csv")

### ----------------------------------------------------------
### 2. CLEAN ID FIELDS — drop any match with missing players
### ----------------------------------------------------------

# Convert to numeric safely
for col in ["winner1", "winner2", "loser1", "loser2"]:
    doub[col] = pd.to_numeric(doub[col], errors="coerce")

# Drop rows where ANY doubles player is missing or invalid
doub = doub.dropna(subset=["winner1", "winner2", "loser1", "loser2"])
doub = doub[(doub["winner1"] > 0) & (doub["winner2"] > 0) &
            (doub["loser1"]  > 0) & (doub["loser2"]  > 0)]

# Convert to int now that they’re clean
for col in ["winner1", "winner2", "loser1", "loser2"]:
    doub[col] = doub[col].astype(int)


### ----------------------------------------------------------
### 3. SCORE MARGIN (same method as singles)
### ----------------------------------------------------------

def score_to_margin(row):
    total = 0
    for col in ["set1", "set2", "set3"]:
        val = row[col]
        if isinstance(val, str) and "*" in val:
            a, b = val.split("*")
            total += (int(a) - int(b))
    return total

doub["margin"] = doub.apply(score_to_margin, axis=1)


### ----------------------------------------------------------
### 4. FLIGHT WEIGHTS (same pattern as singles)
### ----------------------------------------------------------

flight_weight = {"D1": 1.30, "D2": 1.15, "D3": 1.05, "D4": 1.00}
doub["flight_weight"] = doub["flight"].map(flight_weight).fillna(1.00)


### ----------------------------------------------------------
### 5. INITIALIZE RATINGS
### ----------------------------------------------------------

# Collect all individual doubles players
all_players = pd.unique(
    doub[["winner1", "winner2", "loser1", "loser2"]].values.ravel("K")
)

ratings = {pid: 20.0 for pid in all_players}   # neutral starting point


### ----------------------------------------------------------
### 6. ELO EXPECTED SCORE FUNCTION
### ----------------------------------------------------------

def expected_score(rA, rB):
    # divisor=12 used because doubles spreads are smaller
    return 1 / (1 + 10**((rB - rA) / 12))

K = 6.0


### ----------------------------------------------------------
### 7. ITERATIVE TRAINING LOOP FOR DOUBLES
### ----------------------------------------------------------

for epoch in range(12):
    for _, row in doub.iterrows():

        # Extract players
        w1, w2 = row["winner1"], row["winner2"]
        l1, l2 = row["loser1"],  row["loser2"]
        margin = row["margin"]
        f = row["flight_weight"]

        # Team ratings = sum of individual ratings
        rw = ratings[w1] + ratings[w2]
        rl = ratings[l1] + ratings[l2]

        # Expected team outcomes
        Ew = expected_score(rw, rl)
        El = 1 - Ew

        # Margin factor (same as singles)
        margin_factor = 1 + (abs(margin) * 0.10)

        # Rating update amount
        delta = K * f * margin_factor

        # Update WINNER TEAM
        change_w = delta * (1 - Ew)
        ratings[w1] += change_w / 2
        ratings[w2] += change_w / 2

        # Update LOSER TEAM
        change_l = delta * (0 - El)
        ratings[l1] += change_l / 2
        ratings[l2] += change_l / 2


### ----------------------------------------------------------
### 8. NORMALIZE RATINGS TO RANGE 5–39
### ----------------------------------------------------------

rating_series = pd.Series(ratings)
min_r, max_r = rating_series.min(), rating_series.max()

def scale_5_39(x):
    return 5 + (x - min_r) * (34 / (max_r - min_r))

scaled = rating_series.apply(scale_5_39)


### ----------------------------------------------------------
### 9. BUILD FINAL OUTPUT TABLE
### ----------------------------------------------------------

player_ratings = pd.DataFrame({
    "playerID": scaled.index,
    "rating": scaled.values
}).sort_values("rating", ascending=False)

player_ratings.to_csv(MAIN_DIR + r"\doubles_player_ratings.csv", index=False)

player_ratings.head(20)

,playerID,rating
983,121901,39.000000
78,107511,37.750494
990,124077,37.537987
64,121899,36.214898
300,100762,35.684072
367,106168,35.605240
217,102363,35.492528
9,119281,35.316553
74,103807,35.250399
371,165255,34.464865


In [8]:
import pandas as pd

BASE = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite"

# Load ratings files
sing_ratings = pd.read_csv(BASE + r"\player_ratings.csv")
doub_ratings = pd.read_csv(BASE + r"\doubles_player_ratings.csv")

# Load players file
players = pd.read_csv(BASE + r"\girls_players.csv")[["playerID", "fullname"]]

# Convert IDs to numeric consistently
sing_ratings["playerID"] = pd.to_numeric(sing_ratings["playerID"], errors="coerce")
doub_ratings["playerID"] = pd.to_numeric(doub_ratings["playerID"], errors="coerce")
players["playerID"] = pd.to_numeric(players["playerID"], errors="coerce")

# Merge names into ratings
sing_with_names = sing_ratings.merge(players, on="playerID", how="left")
doub_with_names = doub_ratings.merge(players, on="playerID", how="left")

# Sort and show top 100
top100_sing = sing_with_names.sort_values("rating", ascending=False).head(100)
top100_doub = doub_with_names.sort_values("rating", ascending=False).head(100)

print("\n=== TOP 100 SINGLES PLAYERS ===\n")
print(top100_sing)

print("\n=== TOP 100 DOUBLES PLAYERS ===\n")
print(top100_doub)



=== TOP 100 SINGLES PLAYERS ===

    playerID     rating              fullname
0     163631  39.000000              Leah Lup
1     123420  37.108171           Raegan Farm
2     145172  37.057582          Kate Bonetto
3     164183  36.752026  Nikhitha Raji Suresh
4     141982  36.271394        Sofia Sorokina
..       ...        ...                   ...
95    166059  24.559409        Kaylee Schauer
96    101025  24.525517    Bridget Szczesniak
97    102363  24.437810         Kendall Sales
98    101258  24.431770        Natalia Temkin
99    102051  24.339195          Emma Huberty

[100 rows x 3 columns]

=== TOP 100 DOUBLES PLAYERS ===

    playerID     rating           fullname
0     121901  39.000000         Lucy Hasle
1     107511  37.750494     Tessa Rozendal
2     124077  37.537987     Malavika Rahul
3     121899  36.214898     Sabrina Larsen
4     100762  35.684072  Aleksandra Vitruk
..       ...        ...                ...
95    142944  27.606315     Kaitlyn Schwin
96    109495